## imports

In [13]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras import Input
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU

# Carga de imagenes a memoria

In [24]:
dirname = os.path.join(os.getcwd(), 'imagenes')
imgpath = dirname + os.sep

images = []
directories = []
dircount = []
prevRoot=r'c:\Users\Loren\Desktop\Facu\Inteligencia Artificial\visual_IA\Redes_convolucionales\imagenes\basket'
cant=0

a=os.walk(imgpath)
print(a,"----------------------")
print("leyendo imagenes de ",imgpath)


# for root, dirnames, filenames in os.walk(imgpath):
#     print(root)
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

<generator object _walk at 0x000001ADC7141740> ----------------------
leyendo imagenes de  c:\Users\Loren\Desktop\Facu\Inteligencia Artificial\visual_IA\Redes_convolucionales\imagenes\
c:\Users\Loren\Desktop\Facu\Inteligencia Artificial\visual_IA\Redes_convolucionales\imagenes\beisball 8824


KeyboardInterrupt: 

# Etiquetas y clases

In [15]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1

y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  68305
0 beisball
1 boxeo
2 ciclismo
3 f1
4 futbol
5 golf
6 natacion
7 tenis
8 americano
Total number of outputs :  9
Output classes :  [0 1 2 3 4 5 6 7 8]


In [16]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

ValueError: Found input variables with inconsistent numbers of samples: [77128, 68305]

In [ ]:
INIT_LR = 1e-3
epochs = 6
batch_size = 64

sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))

sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5)) 
sport_model.add(Dense(nClasses, activation='softmax'))

sport_model.summary()

sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 21, 28, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 21, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 14, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4928)              0         
                                                                 
 dense (Dense)               (None, 32)                157728    
                                                        

C:\Users\Loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizers\optimizer_v2\adagrad.py:81: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
sport_train_dropout = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
sport_model.save("sports_mnist.h5py")

Epoch 1/6
678/678 [==============================] - 12s 16ms/step - loss: 2.0018 - accuracy: 0.2598 - val_loss: 1.6927 - val_accuracy: 0.5523
Epoch 2/6
678/678 [==============================] - 11s 17ms/step - loss: 1.6754 - accuracy: 0.4192 - val_loss: 1.3724 - val_accuracy: 0.6973
Epoch 3/6
678/678 [==============================] - 11s 17ms/step - loss: 1.4561 - accuracy: 0.5132 - val_loss: 1.1614 - val_accuracy: 0.7513
Epoch 4/6
678/678 [==============================] - 11s 17ms/step - loss: 1.3051 - accuracy: 0.5761 - val_loss: 1.0222 - val_accuracy: 0.7852
Epoch 5/6
678/678 [==============================] - 11s 16ms/step - loss: 1.1946 - accuracy: 0.6173 - val_loss: 0.9155 - val_accuracy: 0.8144
Epoch 6/6
678/678 [==============================] - 11s 16ms/step - loss: 1.1147 - accuracy: 0.6456 - val_loss: 0.8399 - val_accuracy: 0.8220


INFO:tensorflow:Assets written to: sports_mnist.h5py\assets


INFO:tensorflow:Assets written to: sports_mnist.h5py\assets


In [ ]:
test_eval = sport_model.evaluate(test_X, test_Y_one_hot, verbose=1)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

424/424 [==============================] - 1s 3ms/step - loss: 0.8252 - accuracy: 0.8257
Test loss: 0.8252356648445129
Test accuracy: 0.8256860375404358
